# Import Packages

In [ ]:
import pandas as pd
import numpy as np
import sklearn.linear_model as lm
from statsmodels.stats.weightstats import ttest_ind
from scipy import stats
from google.colab import drive
import statsmodels.api as sm
import statsmodels.formula.api as smf
from datetime import datetime
from sklearn.utils import resample
from random import choices
from statsmodels.stats.multitest import multipletests

In [ ]:
try:
  from stargazer.stargazer import Stargazer
except ImportError:
  %pip install stargazer
  from stargazer.stargazer import Stargazer

# Import data

In [ ]:
# This is Nabeel, I renamed the file path in these cells so i could access them from my drive.
baseline = pd.read_stata('/content/drive/MyDrive/project_3_econ148/data/baseline.dta')
bok_inflation = pd.read_stata('/content/drive/MyDrive/project_3_econ148/data/BOK_inflation.dta')
cleanpricedata_y1y2 = pd.read_stata('/content/drive/MyDrive/project_3_econ148/data/cleanPriceData_Y1Y2.dta')
intensity_obs_short = pd.read_stata('/content/drive/MyDrive/project_3_econ148/data/intensity_obs_short.dta')
lrfu_select_dataset = pd.read_stata('/content/drive/MyDrive/project_3_econ148/data/LRFU_select_dataset.dta')
ms1ms2_pooled = pd.read_stata('/content/drive/MyDrive/project_3_econ148/data/MS1MS2_pooled.dta')
repayment_datay1 = pd.read_stata('/content/drive/MyDrive/project_3_econ148/data/repayment_dataY1.dta')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/project_3_econ148/data/baseline.dta'

In [ ]:
# mount google drive if not already mounted
drive.mount('/content/drive')

# data should be stored in an folder directly under the main drive such that the files are store under '/content/drive/MyDrive/project_3_econ148/data/' for it to import the data correctly
baseline = pd.read_stata('/content/drive/MyDrive/project_3_econ148/data/baseline.dta')
bok_inflation = pd.read_stata('/content/drive/MyDrive/project_3_econ148/data/BOK_inflation.dta')
cleanpricedata_y1y2 = pd.read_stata('/content/drive/MyDrive/project_3_econ148/data/cleanPriceData_Y1Y2.dta')
intensity_obs_short = pd.read_stata('/content/drive/MyDrive/project_3_econ148/data/intensity_obs_short.dta')
lrfu_select_dataset = pd.read_stata('/content/drive/MyDrive/project_3_econ148/data/LRFU_select_dataset.dta')
ms1ms2_pooled = pd.read_stata('/content/drive/MyDrive/project_3_econ148/data/MS1MS2_pooled.dta')
repayment_datay1 = pd.read_stata('/content/drive/MyDrive/project_3_econ148/data/repayment_dataY1.dta')

Mounted at /content/drive


# Recreating the tables from the paper

## Table 1

We start by cleaning the data

In [ ]:
# clean ms1ms2_pooled (drop if MS !=2, keep columns oafid and treatMS1MS2, group by oafid and take mean and rename)
ms1ms2_pooled_tab1 = ms1ms2_pooled[ms1ms2_pooled['MS']==2]
ms1ms2_pooled_tab1 = ms1ms2_pooled_tab1[['oafid', 'treatMS1MS2']]
ms1ms2_pooled_tab1 = ms1ms2_pooled_tab1.groupby('oafid', as_index=False).mean()
ms1ms2_pooled_tab1.rename(columns={'treatMS1MS2': 'treat13'}, inplace=True)
print(ms1ms2_pooled_tab1.shape[0]) # checking we have the right number of observations as described in the original article

1019


In [ ]:
# clean baseline data (the stata code indicates that the variables columns 'businessprofitmonth' and 'delta' should be kept, however they have already been renamed to 'businessprofitmonth_base' and 'delta_base')
base_cols = ['oafid', 'logtotcons_base', 'male', 'num_adults', 'num_schoolchildren', 'finished_primary',
                   'finished_secondary', 'cropland', 'num_rooms', 'schoolfees', 'totcons_base', 'logpercapcons_base',
                   'total_cash_savings_base', 'total_cash_savings_trimmed', 'has_savings_acct', 'taken_bank_loan',
                   'taken_informal_loan', 'liquidWealth', 'wagepay', 'businessprofitmonth_base', 'price_avg_diff_pct',
                   'price_expect_diff_pct', 'harvest2011', 'netrevenue2011', 'netseller2011', 'autarkic2011',
                   'maizelostpct2011', 'harvest2012', 'correct_interest', 'digit_recall', 'maizegiver', 'delta_base', 'treatment']
baseline_clean = baseline[base_cols].copy()

# rename columns
baseline_clean.columns = [col + '_base' if not col.endswith('_base') and col != 'oafid' and col != 'treatment' else col for col in baseline_clean.columns]
baseline_clean.rename(columns={'treatment': 'treatment2012'}, inplace=True)

# generate treat12 as bool for treatment and control in 2012
baseline_clean['treat12'] = baseline_clean['treatment2012'].apply(lambda x: x in ['T1', 'T2'])
baseline_clean.loc[baseline_clean['treatment2012'] == '', 'treat12'] = np.nan

In [ ]:
# merge baseline_clean and ms1ms2_pooled_clean on oafid
base_ms1ms2_pool = pd.merge(baseline_clean, ms1ms2_pooled_tab1, on='oafid', how='left')

In [ ]:
# create table 1
# copy in case we need this later
df_tab1 = base_ms1ms2_pool.copy()
df_tab1['schoolfees_base'] = df_tab1['schoolfees_base']*1000

# var list for table 1
vars_list = [
    "male_base", "num_adults_base", "num_schoolchildren_base", "finished_primary_base",
    "finished_secondary_base", "cropland_base", "num_rooms_base", "schoolfees_base",
    "totcons_base", "logpercapcons_base", "total_cash_savings_base",
    "total_cash_savings_trimmed_base", "has_savings_acct_base", "taken_bank_loan_base",
    "taken_informal_loan_base", "liquidWealth_base", "wagepay_base",
    "businessprofitmonth_base", "price_avg_diff_pct_base",
    "price_expect_diff_pct_base", "harvest2011_base", "netrevenue2011_base",
    "netseller2011_base", "autarkic2011_base", "maizelostpct2011_base",
    "harvest2012_base", "correct_interest_base", "digit_recall_base",
    "maizegiver_base"
]

renaming = {
    "male_base": "Male",
    "num_adults_base": "Number of adults",
    "num_schoolchildren_base": "Children in school",
    "finished_primary_base": "Finished primary school",
    "finished_secondary_base": "Finished secondary school",
    "cropland_base": "Total cropland (acres)",
    "num_rooms_base": "Number of rooms in household",
    "schoolfees_base": "Total school fees",
    "totcons_base": "Average monthly consumption (Ksh)",
    "logpercapcons_base": "Average monthly consumption/capita (log)",
    "total_cash_savings_base": "Total cash savings (Ksh)",
    "total_cash_savings_trimmed_base": "Total cash savings (trim)",
    "has_savings_acct_base": "Has bank savings acct",
    "taken_bank_loan_base": "Taken bank loan",
    "taken_informal_loan_base": "Taken informal loan",
    "liquidWealth_base": "Liquid wealth (Ksh)",
    "wagepay_base": "Off-farm wages (Ksh)",
    "businessprofitmonth_base": "Business profit (Ksh)",
    "price_avg_diff_pct_base": "Avg $\%\Delta$ price Sep-Jun",
    "price_expect_diff_pct_base": "Expect $\%\Delta$ price Sep12-Jun13",
    "harvest2011_base": "2011 LR harvest (bags)",
    "netrevenue2011_base": "Net revenue 2011 (Ksh)",
    "netseller2011_base": "Net seller 2011",
    "autarkic2011_base": "Autarkic 2011",
    "maizelostpct2011_base": "\% maize lost 2011",
    "harvest2012_base": "2012 LR harvest (bags)",
    "correct_interest_base": "Calculated interest correctly",
    "digit_recall_base": "Digit span recall",
    "maizegiver_base": "Maize giver"
}

# function to perform t-tests
def t_test_by_group(df, var, group_var='treat12'):
    group1 = df[df[group_var] == 0][var].dropna()
    group2 = df[df[group_var] == 1][var].dropna()
    t_stat, p_val = stats.ttest_ind(group1, group2, equal_var=False)
    return group1.mean(), group2.mean(), len(group1) + len(group2), t_stat, p_val

# applying t-tests and collecting results
results = []
for var in vars_list:
    control_mean, treat_mean, obs, t_stat, p_val = t_test_by_group(df_tab1, var)
    std_diff = (treat_mean - control_mean) / np.sqrt(((len(df_tab1[df_tab1['treat12'] == 0][var]) - 1) * np.std(df_tab1[df_tab1['treat12'] == 0][var], ddof=1) ** 2 + (len(df_tab1[df_tab1['treat12'] == 1][var]) - 1) * np.std(df_tab1[df_tab1['treat12'] == 1][var], ddof=1) ** 2) / (len(df_tab1[df_tab1['treat12'] == 0][var]) + len(df_tab1[df_tab1['treat12'] == 1][var]) - 2))
    results.append([var, treat_mean, control_mean, obs, std_diff, p_val])

# convert results to a df to use pandas output to latex
results_df = pd.DataFrame(results, columns=['Variable', 'Treat Mean', 'Control Mean', 'Observations', 'Std Diff', 'P-value'])
results_df['Variable'] = results_df['Variable'].map(renaming)
results_df = results_df.rename(columns={
    'Variable':'Baseline characteristic',
    'Treat Mean':'Treat',
    'Control Mean':'Control',
    'Observations':'Obs',
    'Std Diff':'Std diff',
    'P-value':'P-val'})

latex_table1 = results_df.to_latex(index=False, float_format="%.3f")
print(latex_table1)

\begin{tabular}{lrrrrr}
\toprule
Baseline characteristic & Treat & Control & Obs & Std diff & P-val \\
\midrule
Male & 0.296 & 0.334 & 1589 & -0.083 & 0.109 \\
Number of adults & 3.004 & 3.196 & 1510 & -0.099 & 0.067 \\
Children in school & 2.998 & 3.072 & 1589 & -0.038 & 0.454 \\
Finished primary school & 0.718 & 0.772 & 1490 & -0.122 & 0.019 \\
Finished secondary school & 0.253 & 0.270 & 1490 & -0.039 & 0.460 \\
Total cropland (acres) & 2.441 & 2.398 & 1512 & 0.014 & 0.796 \\
Number of rooms in household & 3.073 & 3.252 & 1511 & -0.072 & 0.219 \\
Total school fees & 27239.693 & 29813.631 & 1589 & -0.068 & 0.191 \\
Average monthly consumption (Ksh) & 14970.862 & 15371.378 & 1437 & -0.032 & 0.550 \\
Average monthly consumption/capita (log) & 7.975 & 7.963 & 1434 & 0.019 & 0.721 \\
Total cash savings (Ksh) & 5157.396 & 8021.499 & 1572 & -0.128 & 0.028 \\
Total cash savings (trim) & 4731.623 & 5389.836 & 1572 & -0.050 & 0.343 \\
Has bank savings acct & 0.419 & 0.425 & 1589 & -0.012 & 0.8

## Table 2

In [ ]:
# year 1 overall
ms1ms2_clean1 = ms1ms2_pooled.loc[:, ['treat12', 'interviewdate1', 'interviewdate', 'Y1round1', 'Y1round2', 'Y1round3', 'treatMS1MS2', 'inventory_trim', 'groupnum', 'strata_group', 'round']].dropna()
ms1ms2_clean1['inter_R1'] = ms1ms2_clean1['Y1round1'] * ms1ms2_clean1['treat12']
ms1ms2_clean1['inter_R2'] = ms1ms2_clean1['Y1round2'] * ms1ms2_clean1['treat12']
ms1ms2_clean1['inter_R3'] = ms1ms2_clean1['Y1round3'] * ms1ms2_clean1['treat12']

# model specification with fixed effects for strata_group
model = smf.ols('inventory_trim ~ treat12 + interviewdate + C(strata_group)', data=ms1ms2_clean1)

# fitting the model with robust standard errors clustered by 'groupnum'
results1 = model.fit(cov_type='cluster', cov_kwds={'groups': ms1ms2_clean1['groupnum']})

# extract necessary statistics
model_params1 = results1.params
model_pvalues1 = results1.pvalues
mean_dv1 = ms1ms2_clean1['inventory_trim'].mean()
sd_dv1 = ms1ms2_clean1['inventory_trim'].std()

# calculating the Bonferroni corrections for the p-values
bonferroni_pvalues1 = {key: multipletests(results1.pvalues[key], method='fdr_bh')[1] for key, val in model_pvalues1.items() if key in ['treat12', 'inter_R1', 'inter_R2', 'inter_R3']}

In [ ]:
# year 1 by round
model_interactions = smf.ols('inventory_trim ~ inter_R1 + inter_R2 + inter_R3 + interviewdate + C(Y1round1) + C(Y1round2) + C(Y1round3) + C(strata_group)', data=ms1ms2_clean1)
results2 = model_interactions.fit(cov_type='cluster', cov_kwds={'groups': ms1ms2_clean1['groupnum']})

# extract necessary statistics
model_params2 = results2.params
model_pvalues2 = results2.pvalues
mean_dv2 = ms1ms2_clean1['inventory_trim'].mean()
sd_dv2 = ms1ms2_clean1['inventory_trim'].std()

# number of tests, include only treat12 and its interaction terms if they exist
num_tests = 1 + ('inter_R1' in model_params2) + ('inter_R2' in model_params2) + ('inter_R3' in model_params2)

# calculating the Bonferroni corrections for the p-values
bonferroni_pvalues2 = {key: multipletests(results2.pvalues[key], method='fdr_bh')[1] for key, val in model_pvalues2.items() if key in ['treat12', 'inter_R1', 'inter_R2', 'inter_R3']}

In [ ]:
# year 2 overall
ms1ms2_clean2 = ms1ms2_pooled.loc[:, ['treat13', 'Y2round1', 'Y2round2', 'Y2round3', 'treatMS1MS2', 'inventory_trim', 'interviewdate', 'date', 'strata_group', 'groupnum']].dropna()
ms1ms2_clean2['inter_R1'] = ms1ms2_clean2['Y2round1'] * ms1ms2_clean2['treat13']
ms1ms2_clean2['inter_R2'] = ms1ms2_clean2['Y2round2'] * ms1ms2_clean2['treat13']
ms1ms2_clean2['inter_R3'] = ms1ms2_clean2['Y2round3'] * ms1ms2_clean2['treat13']

# model specification with fixed effects for strata_group
model = smf.ols('inventory_trim ~ treat13 + interviewdate + C(strata_group)', data=ms1ms2_clean2)

# Fitting the model with robust standard errors clustered by 'groupnum'
results3 = model.fit(cov_type='cluster', cov_kwds={'groups': ms1ms2_clean2['groupnum']})

# extract necessary statistics
model_params3 = results3.params
model_pvalues3 = results3.pvalues
mean_dv3 = ms1ms2_clean2['inventory_trim'].mean()
sd_dv3 = ms1ms2_clean2['inventory_trim'].std()

# number of tests, include only treat12 and its interaction terms if they exist
num_tests = 1 + ('inter_R1' in model_params3) + ('inter_R2' in model_params3) + ('inter_R3' in model_params3)

# calculating the Bonferroni corrections for the p-values
bonferroni_pvalues3 = {key: multipletests(results3.pvalues[key], method='fdr_bh')[1] for key, val in model_pvalues3.items() if key in ['treat13', 'inter_R1', 'inter_R2', 'inter_R3']}

In [ ]:
# year 2 by round
model_interactions = smf.ols('inventory_trim ~ inter_R1 + inter_R2 + inter_R3 + interviewdate + C(Y2round1) + C(Y2round2) + C(Y2round3) + C(strata_group)', data=ms1ms2_clean2)
results4 = model_interactions.fit(cov_type='cluster', cov_kwds={'groups': ms1ms2_clean2['groupnum']})

# extract necessary statistics
model_params4 = results4.params
model_pvalues4 = results4.pvalues
mean_dv4 = ms1ms2_clean2['inventory_trim'].mean()
sd_dv4 = ms1ms2_clean2['inventory_trim'].std()

# number of tests, include only treat12 and its interaction terms if they exist
num_tests = 1 + ('inter_R1' in model_params4) + ('inter_R2' in model_params4) + ('inter_R3' in model_params4)

# calculating the Bonferroni corrections for the p-values
bonferroni_pvalues4 = {key: multipletests(results4.pvalues[key], method='fdr_bh')[1] for key, val in model_pvalues4.items() if key in ['treat13', 'inter_R1', 'inter_R2', 'inter_R3']}

In [ ]:
# pooled overall
ms1ms2_clean3 = pd.concat([ms1ms2_clean1, ms1ms2_clean2], ignore_index=True).fillna(0)
ms1ms2_clean3.sort_values(by='interviewdate')

# model specification with fixed effects for strata_group
model = smf.ols('inventory_trim ~ treatMS1MS2 + interviewdate + C(strata_group)', data=ms1ms2_clean3)

# fitting the model with robust standard errors clustered by 'groupnum'
results5 = model.fit(cov_type='cluster', cov_kwds={'groups': ms1ms2_clean3['groupnum']})

# extract necessary statistics
model_params5 = results5.params
model_pvalues5 = results5.pvalues
mean_dv5 = ms1ms2_clean3['inventory_trim'].mean()
sd_dv5 = ms1ms2_clean3['inventory_trim'].std()
# number of tests, include only treat12 and its interaction terms if they exist
num_tests = 1 + ('inter_R1' in model_params5) + ('inter_R2' in model_params5) + ('inter_R3' in model_params5)

# calculating the Bonferroni corrections for the p-values
bonferroni_pvalues5 = {key: multipletests(results5.pvalues[key], method='fdr_bh')[1] for key, val in model_pvalues5.items() if key in ['treatMS1MS2', 'inter_R1', 'inter_R2', 'inter_R3']}

In [ ]:
# pooled by round
# model specification with fixed effects for strata_group
model = smf.ols('inventory_trim ~ inter_R1 + inter_R2 + inter_R3 + interviewdate + C(Y1round1) + C(Y1round2) + C(Y1round3) + C(Y2round1) + C(Y2round2) + C(Y2round3) + C(strata_group)', data=ms1ms2_clean3)

# fitting the model with robust standard errors clustered by 'groupnum'
results6 = model.fit(cov_type='cluster', cov_kwds={'groups': ms1ms2_clean3['groupnum']})

# Extract necessary statistics
model_params6 = results6.params
model_pvalues6 = results6.pvalues
mean_dv6 = ms1ms2_clean3['inventory_trim'].mean()
sd_dv6 = ms1ms2_clean3['inventory_trim'].std()

# number of tests, include only treat12 and its interaction terms if they exist
num_tests = 1 + ('inter_R1' in model_params6) + ('inter_R2' in model_params6) + ('inter_R3' in model_params6)

# calculating the Bonferroni corrections for the p-values
bonferroni_pvalues6 = {key: multipletests(results6.pvalues[key], method='fdr_bh')[1] for key, val in model_pvalues6.items() if key in ['treatMS1MS2', 'inter_R1', 'inter_R2', 'inter_R3']}

In [ ]:
# collecting p-values in tables to easily add them to the latex table
results_tab2 = [results1,results2,results3,results4,results5,results6]

# get p-values for treat12, treat13, treatMS1MS2, inter_R1, inter_R2, inter_R3
p_values = {}
for result in results_tab2:
    p_values[result] = np.round(result.pvalues,3)
    p_values_df= pd.DataFrame(p_values)

p_values_df = p_values_df.loc[['treat12', 'treat13', 'treatMS1MS2', 'inter_R1', 'inter_R2', 'inter_R3']]
p_values_df.columns = ['(1)', '(2)', '(3)', '(4)', '(5)', '(6)']
p_values_df.loc['Treat'] = p_values_df.loc[['treat12', 'treat13', 'treatMS1MS2']].mean()
p_values_df.drop(['treat12', 'treat13', 'treatMS1MS2'], inplace=True)

# fwer correction
fwer_p_values = [bonferroni_pvalues1, bonferroni_pvalues2, bonferroni_pvalues3, bonferroni_pvalues4, bonferroni_pvalues5, bonferroni_pvalues6]

# get p-values for treat12, treat13, treatMS1MS2, inter_R1, inter_R2, inter_R3
p_values_fwer = {}
for i, p_values in enumerate(fwer_p_values):
    p_values_fwer[i] = p_values
    p_values_df_fwer = pd.DataFrame(p_values_fwer)

p_values_df_fwer.loc['Treat'] = p_values_df_fwer.loc[['treat12', 'treat13', 'treatMS1MS2']].mean()
p_values_df_fwer.drop(['treat12', 'treat13', 'treatMS1MS2'], inplace=True)

# changeing everything to numeric and rounding
def to_numeric(x):
    if isinstance(x, np.ndarray) and x.size == 1:
        return x.item()  # Converts a one-element array to a scalar
    return x

p_values_df_fwer = p_values_df_fwer.applymap(to_numeric).round(3)

In [ ]:
# creating latex table
stargazer_tab2 = Stargazer(results_tab2)

stargazer_tab2.custom_columns(['Y1', 'Y2','Pooled'], [2,2,2])
stargazer_tab2.significant_digits(3)
stargazer_tab2.rename_covariates({'treat12': 'Treat','treat13': 'Treat', 'treatMS1MS2': 'Treat', 'inter_R1': 'Treat - R1', 'inter_R2': 'Treat - R2', 'inter_R3': 'Treat - R3'})
stargazer_tab2.covariate_order(['treat12', 'treat13', 'treatMS1MS2', 'inter_R1', 'inter_R2', 'inter_R3'])
# adding p-values
stargazer_tab2.add_line('P-Val Treat',p_values_df.loc['Treat'].tolist())
stargazer_tab2.add_line('P-Val Treat FWER',p_values_df_fwer.loc['Treat'].tolist())
stargazer_tab2.add_line('P-Val Treat - R1',p_values_df.loc['inter_R1'].tolist())
stargazer_tab2.add_line('P-Val Treat - R1 FWER',p_values_df_fwer.loc['inter_R1'].tolist())
stargazer_tab2.add_line('P-Val Treat - R2',p_values_df.loc['inter_R2'].tolist())
stargazer_tab2.add_line('P-Val Treat - R2 FWER',p_values_df_fwer.loc['inter_R2'].tolist())
stargazer_tab2.add_line('P-Val Treat - R3',p_values_df.loc['inter_R3'].tolist())
stargazer_tab2.add_line('P-Val Treat - R3 FWER',p_values_df_fwer.loc['inter_R3'].tolist())


latex_table2 = stargazer_tab2.render_latex()

latex_table2 = latex_table2.replace("\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \\",
                                "\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \n \\\ & Overall & By rd & Overall & By rd & Overall & By rd \\")
latex_table2 = latex_table2.replace("Adjusted $R^2$", "% Adjusted $R^2$")
latex_table2 = latex_table2.replace("Residual Std. Error", "% Residual Std. Error")
latex_table2 = latex_table2.replace("F Statistic", "% F Statistic")
latex_table2 = latex_table2.replace("\\textit{Note","% \\textit{Note")
latex_table2 = latex_table2.replace("nan","")
latex_table2 = latex_table2.replace("inventory_trim","Inventory Trim")
latex_table2 = latex_table2.replace("\\begin{table}[!htbp] \\centering", "")
latex_table2 = latex_table2.replace("\\end{table}", "")

print(latex_table2)



\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable: Inventory Trim}} \
\cr \cline{2-7}
\\[-1.8ex] & \multicolumn{2}{c}{Y1} & \multicolumn{2}{c}{Y2} & \multicolumn{2}{c}{Pooled}  \\
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) 
 \\ & Overall & By rd & Overall & By rd & Overall & By rd \\
\hline \\[-1.8ex]
 Treat & 0.574$^{***}$ & & & & & \\
& (0.140) & & & & & \\
 Treat & & & 0.546$^{***}$ & & & \\
& & & (0.129) & & & \\
 Treat & & & & & 0.565$^{***}$ & \\
& & & & & (0.097) & \\
 Treat - R1 & & 0.872$^{***}$ & & 1.242$^{***}$ & & 1.050$^{***}$ \\
& & (0.276) & & (0.235) & & (0.184) \\
 Treat - R2 & & 0.753$^{***}$ & & 0.304$^{*}$ & & 0.546$^{***}$ \\
& & (0.171) & & (0.166) & & (0.120) \\
 Treat - R3 & & 0.111$^{}$ & & 0.082$^{}$ & & 0.094$^{}$ \\
& & (0.083) & & (0.344) & & (0.162) \\
 P-Val Treat & 0.0 &  & 0.0 &  & 0.0 &  \\
 P-Val Treat FWER & 0.0 &  & 0.0 &  & 0.0 &  \\
 P-Val Treat - R1 &  & 0.002 &  &

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 37, but rank is 36
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 37, but rank is 36
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 63, but rank is 62
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 71, but rank is 68
  warnings.warn('covariance of constraints does not have full '


## Table 3

In [ ]:
# year 1 overall
ms1ms2_clean1 = ms1ms2_pooled.loc[:, ['treat12', 'interviewdate1', 'interviewdate', 'Y1round1', 'Y1round2', 'Y1round3', 'treatMS1MS2', 'netrevenue_trim', 'groupnum', 'strata_group', 'round']].dropna()
ms1ms2_clean1['inter_R1'] = ms1ms2_clean1['Y1round1'] * ms1ms2_clean1['treat12']
ms1ms2_clean1['inter_R2'] = ms1ms2_clean1['Y1round2'] * ms1ms2_clean1['treat12']
ms1ms2_clean1['inter_R3'] = ms1ms2_clean1['Y1round3'] * ms1ms2_clean1['treat12']

# Example of running a regression for 'inventory_trim' against 'treat12' with control
# variables and fixed effects for strata_group and clustering by groupnum

# model specification with fixed effects for strata_group
model = smf.ols('netrevenue_trim ~ treat12 + interviewdate + C(strata_group)', data=ms1ms2_clean1)

# fitting the model with robust standard errors clustered by 'groupnum'
results1 = model.fit(cov_type='cluster', cov_kwds={'groups': ms1ms2_clean1['groupnum']})

# extract necessary statistics
model_params1 = results1.params
model_pvalues1 = results1.pvalues
mean_dv1 = ms1ms2_clean1['netrevenue_trim'].mean()
sd_dv1 = ms1ms2_clean1['netrevenue_trim'].std()

# number of tests, include only treat12 and its interaction terms if they exist
num_tests = 1 + ('inter_R1' in model_params1) + ('inter_R2' in model_params1) + ('inter_R3' in model_params1)

# calculating the Bonferroni corrections for the p-values
bonferroni_pvalues1 = {key: multipletests(results1.pvalues[key], method='fdr_bh')[1] for key, val in model_pvalues1.items() if key in ['treat12', 'inter_R1', 'inter_R2', 'inter_R3']}

In [ ]:
# year 1 by rounds
# Similarly, for the models with the interactions (inter_Y1R1, inter_Y1R2, inter_Y1R3):
model_interactions = smf.ols('netrevenue_trim ~ inter_R1 + inter_R2 + inter_R3 + interviewdate + C(Y1round1) + C(Y1round2) + C(Y1round3) + C(strata_group)', data=ms1ms2_clean1)
results2 = model_interactions.fit(cov_type='cluster', cov_kwds={'groups': ms1ms2_clean1['groupnum']})

# extract necessary statistics
model_params2 = results2.params
model_pvalues2 = results2.pvalues
mean_dv2 = ms1ms2_clean1['netrevenue_trim'].mean()
sd_dv2 = ms1ms2_clean1['netrevenue_trim'].std()

# number of tests, include only treat12 and its interaction terms if they exist
num_tests = 1 + ('inter_R1' in model_params2) + ('inter_R2' in model_params2) + ('inter_R3' in model_params2)

# calculating the Bonferroni corrections for the p-values
bonferroni_pvalues2 = {key: multipletests(results2.pvalues[key], method='fdr_bh')[1] for key, val in model_pvalues2.items() if key in ['inter_R1', 'inter_R2', 'inter_R3']}

In [ ]:
# year 2 overall
ms1ms2_clean2 = ms1ms2_pooled.loc[:, ['treat13', 'Y2round1', 'Y2round2', 'Y2round3', 'treatMS1MS2', 'netrevenue_trim', 'interviewdate', 'date', 'strata_group', 'groupnum']].dropna()
ms1ms2_clean2['inter_R1'] = ms1ms2_clean2['Y2round1'] * ms1ms2_clean2['treat13']
ms1ms2_clean2['inter_R2'] = ms1ms2_clean2['Y2round2'] * ms1ms2_clean2['treat13']
ms1ms2_clean2['inter_R3'] = ms1ms2_clean2['Y2round3'] * ms1ms2_clean2['treat13']

# model specification with fixed effects for strata_group
model = smf.ols('netrevenue_trim ~ treat13 + interviewdate + C(strata_group)', data=ms1ms2_clean2)

# fitting the model with robust standard errors clustered by 'groupnum'
results3 = model.fit(cov_type='cluster', cov_kwds={'groups': ms1ms2_clean2['groupnum']})

# extract necessary statistics
model_params3 = results3.params
model_pvalues3 = results3.pvalues
mean_dv3 = ms1ms2_clean2['netrevenue_trim'].mean()
sd_dv3 = ms1ms2_clean2['netrevenue_trim'].std()

# number of tests, include only treat12 and its interaction terms if they exist
num_tests = 1 + ('inter_R1' in model_params3) + ('inter_R2' in model_params3) + ('inter_R3' in model_params3)

# calculating the Bonferroni corrections for the p-values
bonferroni_pvalues3 = {key: multipletests(results3.pvalues[key], method='fdr_bh')[1] for key, val in model_pvalues3.items() if key in ['treat13', 'inter_R1', 'inter_R2', 'inter_R3']}

In [ ]:
# year 2 by round
# Similarly, for the models with the interactions (inter_Y1R1, inter_Y1R2, inter_Y1R3):
model_interactions = smf.ols('netrevenue_trim ~ inter_R1 + inter_R2 + inter_R3 + interviewdate + C(Y2round1) + C(Y2round2) + C(Y2round3) + C(strata_group)', data=ms1ms2_clean2)
results4 = model_interactions.fit(cov_type='cluster', cov_kwds={'groups': ms1ms2_clean2['groupnum']})
# results4.summary()

# extract necessary statistics
model_params4 = results4.params
model_pvalues4 = results4.pvalues
mean_dv4 = ms1ms2_clean2['netrevenue_trim'].mean()
sd_dv4 = ms1ms2_clean2['netrevenue_trim'].std()

# number of tests, include only treat12 and its interaction terms if they exist
num_tests = 1 + ('inter_R1' in model_params4) + ('inter_R2' in model_params4) + ('inter_R3' in model_params4)

# calculating the Bonferroni corrections for the p-values
bonferroni_pvalues4 = {key: multipletests(results4.pvalues[key], method='fdr_bh')[1] for key, val in model_pvalues4.items() if key in ['treat13', 'inter_R1', 'inter_R2', 'inter_R3']}

In [ ]:
# pooled overall
ms1ms2_clean3 = pd.concat([ms1ms2_clean1, ms1ms2_clean2], ignore_index=True).fillna(0)
ms1ms2_clean3.sort_values(by='interviewdate')

# model specification with fixed effects for strata_group
model = smf.ols('netrevenue_trim ~ treatMS1MS2 + interviewdate + C(strata_group)', data=ms1ms2_clean3)

# fitting the model with robust standard errors clustered by 'groupnum'
results5 = model.fit(cov_type='cluster', cov_kwds={'groups': ms1ms2_clean3['groupnum']})

# extract necessary statistics
model_params5 = results5.params
model_pvalues5 = results5.pvalues
mean_dv5 = ms1ms2_clean3['netrevenue_trim'].mean()
sd_dv5 = ms1ms2_clean3['netrevenue_trim'].std()

# number of tests, include only treat12 and its interaction terms if they exist
num_tests = 1 + ('inter_R1' in model_params5) + ('inter_R2' in model_params5) + ('inter_R3' in model_params5)

# calculating the Bonferroni corrections for the p-values
bonferroni_pvalues5 = {key: multipletests(results5.pvalues[key], method='fdr_bh')[1] for key, val in model_pvalues5.items() if key in ['treatMS1MS2', 'inter_R1', 'inter_R2', 'inter_R3']}

In [ ]:
# pooled by round
# model specification with fixed effects for strata_group
model = smf.ols('netrevenue_trim ~ inter_R1 + inter_R2 + inter_R3 + interviewdate + C(Y1round1) + C(Y1round2) + C(Y1round3) + C(Y2round1) + C(Y2round2) + C(Y2round3) + C(strata_group)', data=ms1ms2_clean3)

# fitting the model with robust standard errors clustered by 'groupnum'
results6 = model.fit(cov_type='cluster', cov_kwds={'groups': ms1ms2_clean3['groupnum']})

# extract necessary statistics
model_params6 = results6.params
model_pvalues6 = results6.pvalues
mean_dv6 = ms1ms2_clean3['netrevenue_trim'].mean()
sd_dv6 = ms1ms2_clean3['netrevenue_trim'].std()

# number of tests, include only treat12 and its interaction terms if they exist
num_tests = 1 + ('inter_R1' in model_params6) + ('inter_R2' in model_params6) + ('inter_R3' in model_params6)

# calculating the Bonferroni corrections for the p-values
bonferroni_pvalues6 = {key: multipletests(results6.pvalues[key], method='fdr_bh')[1] for key, val in model_pvalues6.items() if key in ['inter_R1', 'inter_R2', 'inter_R3']}

In [ ]:
# collecting p-values in tables to easily add them to the latex table
results_tab3 = [results1,results2,results3,results4,results5,results6]

# get p-values for treat12, treat13, treatMS1MS2, inter_R1, inter_R2, inter_R3
p_values = {}
for result in results_tab3:
    p_values[result] = np.round(result.pvalues,3)
    p_values_df= pd.DataFrame(p_values)

p_values_df = p_values_df.loc[['treat12', 'treat13', 'treatMS1MS2', 'inter_R1', 'inter_R2', 'inter_R3']]
p_values_df.columns = ['(1)', '(2)', '(3)', '(4)', '(5)', '(6)']
p_values_df.loc['Treat'] = p_values_df.loc[['treat12', 'treat13', 'treatMS1MS2']].mean()
p_values_df.drop(['treat12', 'treat13', 'treatMS1MS2'], inplace=True)

# fwer correction
fwer_p_values = [bonferroni_pvalues1, bonferroni_pvalues2, bonferroni_pvalues3, bonferroni_pvalues4, bonferroni_pvalues5, bonferroni_pvalues6]

# get p-values for treat12, treat13, treatMS1MS2, inter_R1, inter_R2, inter_R3
p_values_fwer = {}
for i, p_values in enumerate(fwer_p_values):
    p_values_fwer[i] = p_values
    p_values_df_fwer = pd.DataFrame(p_values_fwer)

p_values_df_fwer.loc['Treat'] = p_values_df_fwer.loc[['treat12', 'treat13', 'treatMS1MS2']].mean()
p_values_df_fwer.drop(['treat12', 'treat13', 'treatMS1MS2'], inplace=True)


p_values_df_fwer = p_values_df_fwer.applymap(to_numeric).round(3)

In [ ]:
# creating latex table
stargazer_tab3 = Stargazer(results_tab3)

stargazer_tab3.custom_columns(['Y1', 'Y2','Pooled'], [2,2,2])
stargazer_tab3.significant_digits(3)
stargazer_tab3.rename_covariates({'treat12': 'Treat','treat13': 'Treat', 'treatMS1MS2': 'Treat', 'inter_R1': 'Treat - R1', 'inter_R2': 'Treat - R2', 'inter_R3': 'Treat - R3'})
stargazer_tab3.covariate_order(['treat12', 'treat13', 'treatMS1MS2', 'inter_R1', 'inter_R2', 'inter_R3'])
# adding p-values
stargazer_tab3.add_line('P-Val Treat',p_values_df.loc['Treat'].tolist())
stargazer_tab3.add_line('P-Val Treat FWER',p_values_df_fwer.loc['Treat'].tolist())
stargazer_tab3.add_line('P-Val Treat - R1',p_values_df.loc['inter_R1'].tolist())
stargazer_tab3.add_line('P-Val Treat - R1 FWER',p_values_df_fwer.loc['inter_R1'].tolist())
stargazer_tab3.add_line('P-Val Treat - R2',p_values_df.loc['inter_R2'].tolist())
stargazer_tab3.add_line('P-Val Treat - R2 FWER',p_values_df_fwer.loc['inter_R2'].tolist())
stargazer_tab3.add_line('P-Val Treat - R3',p_values_df.loc['inter_R3'].tolist())
stargazer_tab3.add_line('P-Val Treat - R3 FWER',p_values_df_fwer.loc['inter_R3'].tolist())


latex_table3 = stargazer_tab3.render_latex()

latex_table3 = latex_table3.replace("\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \\",
                                "\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \n \\\ & Overall & By rd & Overall & By rd & Overall & By rd \\")
latex_table3 = latex_table3.replace("Adjusted $R^2$", "% Adjusted $R^2$")
latex_table3 = latex_table3.replace("Residual Std. Error", "% Residual Std. Error")
latex_table3 = latex_table3.replace("F Statistic", "% F Statistic")
latex_table3 = latex_table3.replace("\\textit{Note","% \\textit{Note")
latex_table3 = latex_table3.replace("nan","")
latex_table3 = latex_table3.replace("netrevenue_trim","Netrevenue Trim")
latex_table3 = latex_table3.replace("\\begin{table}[!htbp] \\centering", "")
latex_table3 = latex_table3.replace("\\end{table}", "")

print(latex_table3)



\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable: Netrevenue Trim}} \
\cr \cline{2-7}
\\[-1.8ex] & \multicolumn{2}{c}{Y1} & \multicolumn{2}{c}{Y2} & \multicolumn{2}{c}{Pooled}  \\
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) 
 \\ & Overall & By rd & Overall & By rd & Overall & By rd \\
\hline \\[-1.8ex]
 Treat & 263.790$^{}$ & & & & & \\
& (255.661) & & & & & \\
 Treat & & & 854.114$^{***}$ & & & \\
& & & (303.802) & & & \\
 Treat & & & & & 531.358$^{***}$ & \\
& & & & & (196.315) & \\
 Treat - R1 & & -1164.574$^{***}$ & & 16.478$^{}$ & & -613.581$^{**}$ \\
& & (322.956) & & (444.957) & & (271.653) \\
 Treat - R2 & & 509.851$^{}$ & & 1994.923$^{***}$ & & 1187.967$^{***}$ \\
& & (446.928) & & (503.696) & & (337.460) \\
 Treat - R3 & & 1370.344$^{***}$ & & 565.438$^{}$ & & 998.665$^{***}$ \\
& & (412.602) & & (403.307) & & (291.103) \\
 P-Val Treat & 0.302 &  & 0.005 &  & 0.007 &  \\
 P-Val Treat FWER & 0.3

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 37, but rank is 36
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 37, but rank is 36
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 63, but rank is 62
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 71, but rank is 68
  warnings.warn('covariance of constraints does not have full '


## Table 4

In [ ]:
# year 1 overall
ms1ms2_clean1 = ms1ms2_pooled.loc[:, ['treat12', 'interviewdate1', 'interviewdate', 'Y1round1', 'Y1round2', 'Y1round3', 'treatMS1MS2', 'logtotcons_trim', 'groupnum', 'strata_group', 'round']].dropna()
ms1ms2_clean1['inter_R1'] = ms1ms2_clean1['Y1round1'] * ms1ms2_clean1['treat12']
ms1ms2_clean1['inter_R2'] = ms1ms2_clean1['Y1round2'] * ms1ms2_clean1['treat12']
ms1ms2_clean1['inter_R3'] = ms1ms2_clean1['Y1round3'] * ms1ms2_clean1['treat12']

# model specification with fixed effects for strata_group
model = smf.ols('logtotcons_trim ~ treat12 + interviewdate + C(strata_group)', data=ms1ms2_clean1)

# fitting the model with robust standard errors clustered by 'groupnum'
results1 = model.fit(cov_type='cluster', cov_kwds={'groups': ms1ms2_clean1['groupnum']})

# Extract necessary statistics
model_params1 = results1.params
model_pvalues1 = results1.pvalues
mean_dv1 = ms1ms2_clean1['logtotcons_trim'].mean()
sd_dv1 = ms1ms2_clean1['logtotcons_trim'].std()

# Number of tests, include only treat12 and its interaction terms if they exist
num_tests = 1 + ('inter_R1' in model_params1) + ('inter_R2' in model_params1) + ('inter_R3' in model_params1)

# Calculating the Bonferroni corrections for the p-values
bonferroni_pvalues1 = {key: multipletests(results1.pvalues[key], method='fdr_bh')[1] for key, val in model_pvalues1.items() if key in ['treat12', 'inter_R1', 'inter_R2', 'inter_R3']}

In [ ]:
# year 1 by round
model_interactions = smf.ols('logtotcons_trim ~ inter_R1 + inter_R2 + inter_R3 + interviewdate + C(Y1round1) + C(Y1round2) + C(Y1round3) + C(strata_group)', data=ms1ms2_clean1)
results2 = model_interactions.fit(cov_type='cluster', cov_kwds={'groups': ms1ms2_clean1['groupnum']})

# Extract necessary statistics
model_params2 = results2.params
model_pvalues2 = results2.pvalues
mean_dv2 = ms1ms2_clean1['logtotcons_trim'].mean()
sd_dv2 = ms1ms2_clean1['logtotcons_trim'].std()

# Number of tests, include only treat12 and its interaction terms if they exist
num_tests = 1 + ('inter_R1' in model_params2) + ('inter_R2' in model_params2) + ('inter_R3' in model_params2)

# Calculating the Bonferroni corrections for the p-values
bonferroni_pvalues2 = {key: multipletests(results2.pvalues[key], method='fdr_bh')[1] for key, val in model_pvalues2.items() if key in ['inter_R1', 'inter_R2', 'inter_R3']}

In [ ]:
# year 2 overall
ms1ms2_clean2 = ms1ms2_pooled.loc[:, ['treat13', 'Y2round1', 'Y2round2', 'Y2round3', 'treatMS1MS2', 'logtotcons_trim', 'interviewdate', 'date', 'strata_group', 'groupnum']].dropna()
ms1ms2_clean2['inter_R1'] = ms1ms2_clean2['Y2round1'] * ms1ms2_clean2['treat13']
ms1ms2_clean2['inter_R2'] = ms1ms2_clean2['Y2round2'] * ms1ms2_clean2['treat13']
ms1ms2_clean2['inter_R3'] = ms1ms2_clean2['Y2round3'] * ms1ms2_clean2['treat13']

# model specification with fixed effects for strata_group
model = smf.ols('logtotcons_trim ~ treat13 + interviewdate + C(strata_group)', data=ms1ms2_clean2)

# fitting the model with robust standard errors clustered by 'groupnum'
results3 = model.fit(cov_type='cluster', cov_kwds={'groups': ms1ms2_clean2['groupnum']})

# Extract necessary statistics
model_params3 = results3.params
model_pvalues3 = results3.pvalues
mean_dv3 = ms1ms2_clean2['logtotcons_trim'].mean()
sd_dv3 = ms1ms2_clean2['logtotcons_trim'].std()

# number of tests, include only treat12 and its interaction terms if they exist
num_tests = 1 + ('inter_R1' in model_params3) + ('inter_R2' in model_params3) + ('inter_R3' in model_params3)

# calculating the Bonferroni corrections for the p-values
bonferroni_pvalues3 = {key: multipletests(results3.pvalues[key], method='fdr_bh')[1] for key, val in model_pvalues3.items() if key in ['treat13', 'inter_R1', 'inter_R2', 'inter_R3']}

In [ ]:
# year 2 by round
model_interactions = smf.ols('logtotcons_trim ~ inter_R1 + inter_R2 + inter_R3 + interviewdate + C(Y2round1) + C(Y2round2) + C(Y2round3) + C(strata_group)', data=ms1ms2_clean2)
results4 = model_interactions.fit(cov_type='cluster', cov_kwds={'groups': ms1ms2_clean2['groupnum']})

# extract necessary statistics
model_params4 = results4.params
model_pvalues4 = results4.pvalues
mean_dv4 = ms1ms2_clean2['logtotcons_trim'].mean()
sd_dv4 = ms1ms2_clean2['logtotcons_trim'].std()

# number of tests, include only treat12 and its interaction terms if they exist
num_tests = 1 + ('inter_R1' in model_params4) + ('inter_R2' in model_params4) + ('inter_R3' in model_params4)

# calculating the Bonferroni corrections for the p-values
bonferroni_pvalues4 = {key: multipletests(results4.pvalues[key], method='fdr_bh')[1] for key, val in model_pvalues4.items() if key in ['treat13', 'inter_R1', 'inter_R2', 'inter_R3']}

In [ ]:
# pooled overall
ms1ms2_clean3 = pd.concat([ms1ms2_clean1, ms1ms2_clean2], ignore_index=True).fillna(0)
ms1ms2_clean3.sort_values(by='interviewdate')

# model specification with fixed effects for strata_group
model = smf.ols('logtotcons_trim ~ treatMS1MS2 + interviewdate + C(strata_group)', data=ms1ms2_clean3)

# fitting the model with robust standard errors clustered by 'groupnum'
results5 = model.fit(cov_type='cluster', cov_kwds={'groups': ms1ms2_clean3['groupnum']})


# extract necessary statistics
model_params5 = results5.params
observations5 = len(ms1ms2_clean3)
mean_dv5 = ms1ms2_clean3['logtotcons_trim'].mean()
sd_dv5 = ms1ms2_clean3['logtotcons_trim'].std()

# number of tests, include only treat12 and its interaction terms if they exist
num_tests = 1 + ('inter_R1' in model_params5) + ('inter_R2' in model_params5) + ('inter_R3' in model_params5)

# calculating the Bonferroni corrections for the p-values
bonferroni_pvalues5 = {key: multipletests(results5.pvalues[key], method='fdr_bh')[1] for key, val in model_pvalues5.items() if key in ['treatMS1MS2', 'inter_R1', 'inter_R2', 'inter_R3']}

In [ ]:
# pooled by round
# model specification with fixed effects for strata_group
model = smf.ols('logtotcons_trim ~ inter_R1 + inter_R2 + inter_R3 + interviewdate + C(Y1round1) + C(Y1round2) + C(Y1round3) + C(Y2round1) + C(Y2round2) + C(Y2round3) + C(strata_group)', data=ms1ms2_clean3)

# fitting the model with robust standard errors clustered by 'groupnum'
results6 = model.fit(cov_type='cluster', cov_kwds={'groups': ms1ms2_clean3['groupnum']})

# extract necessary statistics
model_params6 = results6.params
model_pvalues6 = results6.pvalues
mean_dv6 = ms1ms2_clean3['logtotcons_trim'].mean()
sd_dv6 = ms1ms2_clean3['logtotcons_trim'].std()

# number of tests, include only treat12 and its interaction terms if they exist
num_tests = 1 + ('inter_R1' in model_params6) + ('inter_R2' in model_params6) + ('inter_R3' in model_params6)

# calculating the Bonferroni corrections for the p-values
bonferroni_pvalues6 = {key: multipletests(results6.pvalues[key], method='fdr_bh')[1] for key, val in model_pvalues6.items() if key in ['inter_R1', 'inter_R2', 'inter_R3']}

In [ ]:
# collecting p-values in tables to easily add them to the latex table
results_tab4 = [results1,results2,results3,results4,results5,results6]

# get p-values for treat12, treat13, treatMS1MS2, inter_R1, inter_R2, inter_R3
p_values = {}
for result in results_tab4:
    p_values[result] = np.round(result.pvalues,3)
    p_values_df= pd.DataFrame(p_values)

p_values_df = p_values_df.loc[['treat12', 'treat13', 'treatMS1MS2', 'inter_R1', 'inter_R2', 'inter_R3']]
p_values_df.columns = ['(1)', '(2)', '(3)', '(4)', '(5)', '(6)']
p_values_df.loc['Treat'] = p_values_df.loc[['treat12', 'treat13', 'treatMS1MS2']].mean()
p_values_df.drop(['treat12', 'treat13', 'treatMS1MS2'], inplace=True)

# fwer correction
fwer_p_values = [bonferroni_pvalues1, bonferroni_pvalues2, bonferroni_pvalues3, bonferroni_pvalues4, bonferroni_pvalues5, bonferroni_pvalues6]

# get p-values for treat12, treat13, treatMS1MS2, inter_R1, inter_R2, inter_R3
p_values_fwer = {}
for i, p_values in enumerate(fwer_p_values):
    p_values_fwer[i] = p_values
    p_values_df_fwer = pd.DataFrame(p_values_fwer)

p_values_df_fwer.loc['Treat'] = p_values_df_fwer.loc[['treat12', 'treat13', 'treatMS1MS2']].mean()
p_values_df_fwer.drop(['treat12', 'treat13', 'treatMS1MS2'], inplace=True)

# changeing everything to numeric and rounding
p_values_df_fwer = p_values_df_fwer.applymap(to_numeric).round(3)

In [ ]:
# creating latex table
stargazer_tab4 = Stargazer(results_tab4)

stargazer_tab4.custom_columns(['Y1', 'Y2','Pooled'], [2,2,2])
stargazer_tab4.significant_digits(3)
stargazer_tab4.rename_covariates({'treat12': 'Treat','treat13': 'Treat', 'treatMS1MS2': 'Treat', 'inter_R1': 'Treat - R1', 'inter_R2': 'Treat - R2', 'inter_R3': 'Treat - R3'})
stargazer_tab4.covariate_order(['treat12', 'treat13', 'treatMS1MS2', 'inter_R1', 'inter_R2', 'inter_R3'])
# adding p-values
stargazer_tab4.add_line('P-Val Treat',p_values_df.loc['Treat'].tolist())
stargazer_tab4.add_line('P-Val Treat FWER',p_values_df_fwer.loc['Treat'].tolist())
stargazer_tab4.add_line('P-Val Treat - R1',p_values_df.loc['inter_R1'].tolist())
stargazer_tab4.add_line('P-Val Treat - R1 FWER',p_values_df_fwer.loc['inter_R1'].tolist())
stargazer_tab4.add_line('P-Val Treat - R2',p_values_df.loc['inter_R2'].tolist())
stargazer_tab4.add_line('P-Val Treat - R2 FWER',p_values_df_fwer.loc['inter_R2'].tolist())
stargazer_tab4.add_line('P-Val Treat - R3',p_values_df.loc['inter_R3'].tolist())
stargazer_tab4.add_line('P-Val Treat - R3 FWER',p_values_df_fwer.loc['inter_R3'].tolist())


latex_table4 = stargazer_tab4.render_latex()

latex_table4 = latex_table4.replace("\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \\",
                                "\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \n \\\ & Overall & By rd & Overall & By rd & Overall & By rd \\")
latex_table4 = latex_table4.replace("Adjusted $R^2$", "% Adjusted $R^2$")
latex_table4 = latex_table4.replace("Residual Std. Error", "% Residual Std. Error")
latex_table4 = latex_table4.replace("F Statistic", "% F Statistic")
latex_table4 = latex_table4.replace("\\textit{Note","% \\textit{Note")
latex_table4 = latex_table4.replace("nan","")
latex_table4 = latex_table4.replace("logtotcons_trim","Log Total Consumption")
latex_table4 = latex_table4.replace("\\begin{table}[!htbp] \\centering", "")
latex_table4 = latex_table4.replace("\\end{table}", "")

print(latex_table4)



\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable: Log Total Consumption}} \
\cr \cline{2-7}
\\[-1.8ex] & \multicolumn{2}{c}{Y1} & \multicolumn{2}{c}{Y2} & \multicolumn{2}{c}{Pooled}  \\
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) 
 \\ & Overall & By rd & Overall & By rd & Overall & By rd \\
\hline \\[-1.8ex]
 Treat & 0.012$^{}$ & & & & & \\
& (0.030) & & & & & \\
 Treat & & & 0.064$^{*}$ & & & \\
& & & (0.036) & & & \\
 Treat & & & & & 0.036$^{}$ & \\
& & & & & (0.023) & \\
 Treat - R1 & & -0.033$^{}$ & & 0.064$^{}$ & & 0.013$^{}$ \\
& & (0.047) & & (0.047) & & (0.033) \\
 Treat - R2 & & 0.028$^{}$ & & 0.076$^{*}$ & & 0.049$^{*}$ \\
& & (0.039) & & (0.043) & & (0.029) \\
 Treat - R3 & & 0.038$^{}$ & & 0.052$^{}$ & & 0.044$^{}$ \\
& & (0.042) & & (0.047) & & (0.031) \\
 P-Val Treat & 0.683 &  & 0.08 &  & 0.126 &  \\
 P-Val Treat FWER & 0.683 &  & 0.08 &  & 0.126 &  \\
 P-Val Treat - R1 &  & 0.486 &  & 0.1

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 37, but rank is 36
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 37, but rank is 36
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 63, but rank is 62
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 71, but rank is 68
  warnings.warn('covariance of constraints does not have full '


## Table 5

### Clean the data

In [ ]:
ms1ms2_pooled_tab5 = ms1ms2_pooled.copy(deep=True)
max_strata_group = ms1ms2_pooled_tab5['strata_group'].max()
ms1ms2_pooled_tab5.loc[ms1ms2_pooled_tab5['MS'] == 2, 'strata_group'] = ms1ms2_pooled_tab5['groupstrata'] + max_strata_group

ms1ms2_pooled_tab5.loc[ms1ms2_pooled_tab5['MS'] == 2, 'oafid'] = ms1ms2_pooled_tab5['fr_id']

ms1ms2_pooled_tab5['purchasequant2'] = ms1ms2_pooled_tab5['purchasequant']
ms1ms2_pooled_tab5.loc[(ms1ms2_pooled_tab5['purchaseval']==0)&(ms1ms2_pooled_tab5['purchasequant'].isna()),'purchasequant2'] = 0
ms1ms2_pooled_tab5['netsales2'] = ms1ms2_pooled_tab5['salesquant'] - ms1ms2_pooled_tab5['purchasequant2']
ms1ms2_pooled_tab5['netsales'] = ms1ms2_pooled_tab5['netsales2']

ms1ms2_pooled_tab5.drop(columns=['netsales_trim','purchaseval_trim','salesval_trim'], inplace=True)

In [ ]:
# trim outliers
for x in ['purchaseval', 'salesval', 'purchasequant', 'salesquant']:
    quantile = ms1ms2_pooled_tab5[x].quantile([0.99])
    ms1ms2_pooled_tab5[f'{x}_trim'] = ms1ms2_pooled_tab5[x]
    ms1ms2_pooled_tab5.loc[ms1ms2_pooled_tab5[f'{x}_trim'] > quantile[0.99],f'{x}_trim'] = np.nan

quantile = ms1ms2_pooled_tab5['netsales'].quantile([0.005, 0.995])
ms1ms2_pooled_tab5['netsales_trim'] = ms1ms2_pooled_tab5['netsales']
ms1ms2_pooled_tab5.loc[(ms1ms2_pooled_tab5['netsales_trim'] <= quantile[0.005]) | (ms1ms2_pooled_tab5['netsales_trim'] > quantile[0.995]) , 'netsales_trim'] = np.nan

# create id
ms1ms2_pooled_tab5['id'] = ms1ms2_pooled_tab5['oafid'].fillna(ms1ms2_pooled_tab5['fr_id'])

# create effective prices
trim_vars = ['salesquant_trim', 'purchasequant_trim', 'salesval_trim', 'purchaseval_trim']
for var in trim_vars:
    ms1ms2_pooled_tab5[f'tot_{var}'] = ms1ms2_pooled_tab5.groupby(['id', 'MS'])[var].transform('sum')

for x in ['purchase', 'sales']:
    ms1ms2_pooled_tab5[f'effective_{x}_price'] = ms1ms2_pooled_tab5[f'tot_{x}val_trim'] / ms1ms2_pooled_tab5[f'tot_{x}quant_trim']
    ms1ms2_pooled_tab5.loc[ms1ms2_pooled_tab5[f'tot_{x}quant_trim']== 0,f'effective_{x}_price'] = np.nan

### Net Sales

In [ ]:
results = {}

# define variable
dv = 'netsales_trim'
independent_vars = ['z', 'treatMS1MS2_1 + treatMS1MS2_2 + treatMS1MS2_3']

for i, var in enumerate(independent_vars):
    df = ms1ms2_pooled_tab5.copy(deep=True)
    df['z'] = df['treatMS1MS2']
    if var == 'z':
        df.dropna(subset=[dv,'z','interviewdate','Y1round2','Y1round3','Y2round1','Y2round2','Y2round3','strata_group','groupnum'], inplace=True)
    else:
        df.dropna(subset=[dv,'treatMS1MS2_1','treatMS1MS2_2','treatMS1MS2_3','interviewdate','Y1round2','Y1round3','Y2round1','Y2round2','Y2round3','strata_group','groupnum'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    formula = f'{dv} ~ {var} + interviewdate + Y1round2 + Y1round3 + Y2round1 + Y2round2 + Y2round3 + C(strata_group)'
    model = smf.ols(formula, df).fit(cov_type='cluster', cov_kwds={'groups': df['groupnum']})
    mean_dev = df.loc[df['treatMS1MS2'] == 0, dv].mean()
    std_dev = df.loc[df['treatMS1MS2'] == 0, dv].std()
    fwer_pvals = multipletests(model.pvalues, method='fdr_bh')[1]
    results[f'netsales_{i}'] = {'model':model, 'mean_dev':mean_dev, 'std_dev':std_dev, 'fwer_pvals':fwer_pvals}

### Effective Price

In [ ]:
for dv in ['purchase', 'sales']:
    for i, treat in enumerate(['treat12', 'treat13', 'treatMS1MS2']):
        df = ms1ms2_pooled_tab5.copy(deep=True)
        df['z'] = df[treat]
        df = df.drop_duplicates(subset=['id', 'MS'], keep='first')
        df.dropna(subset=[f'effective_{dv}_price','z','groupnum'], inplace=True)
        if treat == 'treatMS1MS2':
            formula = f'effective_{dv}_price ~ z + C(strata_group)'
        else:
            df = df[df['MS'] == i+1]
            formula = f'effective_{dv}_price ~ z + C(strata_group)'
        model = smf.ols(formula, data=df).fit(cov_type='cluster', cov_kwds={'groups': df['groupnum']})
        mean_dev = df.loc[df['z'] == 0, f'effective_{dv}_price'].mean()
        std_dev = df.loc[df['z'] == 0, f'effective_{dv}_price'].std()
        fwer_pvals = multipletests(model.pvalues['z'], method='fdr_bh')[1]
        results[f'{dv}_{treat}'] = {'model':model, 'mean_dev':mean_dev, 'std_dev':std_dev, 'fwer_pvals':fwer_pvals}

### LaTeX output

In [ ]:
models_list = ['netsales_0','netsales_1','purchase_treatMS1MS2','sales_treatMS1MS2']
stargazer = Stargazer([results[model]['model'] for model in models_list])

# get p-values
pvals = np.round(pd.DataFrame({model:results[model]['model'].pvalues for model in ['purchase_treatMS1MS2','sales_treatMS1MS2']}),3)
fwer_pvals = np.round(pd.DataFrame({model:results[model]['fwer_pvals'] for model in ['purchase_treatMS1MS2','sales_treatMS1MS2']}),3)
# rename columns
for i, df in enumerate([pvals, fwer_pvals]):
    df.columns = ['Purchase', 'Sales']
    df['Overall'] = ""
    df['By rd'] = ""
    # reorder columns
    if i == 0:
        pvals = df[['Overall', 'By rd', 'Purchase', 'Sales']]
    else:
        fwer_pvals = df[['Overall', 'By rd', 'Purchase', 'Sales']]

# configure Stargazer object for output
stargazer.custom_columns(['Net Sales', 'Effective Price'], [2, 2])
stargazer.rename_covariates({'z': 'Treat','treatMS1MS2_1':'Treat - R1', 'treatMS1MS2_2':'Treat - R2', 'treatMS1MS2_3':'Treat - R3'})
stargazer.show_degrees_of_freedom(False)
stargazer.significant_digits(3)
stargazer.covariate_order(['z', 'treatMS1MS2_1', 'treatMS1MS2_2', 'treatMS1MS2_3'])
# add p-values as a rows
stargazer.add_line('P-value Treat', pvals.loc['z'].values.tolist())
stargazer.add_line('P-value Treat FWER', fwer_pvals.loc[0].values.tolist())

latex_table5 = stargazer.render_latex()
latex_table5 = latex_table5.replace("\\[-1.8ex] & (1) & (2) & (3) & (4) \\",
                                "\\[-1.8ex] & (1) & (2) & (3) & (4) \n \\\ & Overall & By rd & Purchase & Sales \\")
latex_table5 = latex_table5.replace("Adjusted $R^2$", "% Adjusted $R^2$")
latex_table5 = latex_table5.replace("Residual Std. Error", "% Residual Std. Error")
latex_table5 = latex_table5.replace("F Statistic", "% F Statistic")
latex_table5 = latex_table5.replace("\\textit{Note","% \\textit{Note")
latex_table5 = latex_table5.replace("\\begin{table}[!htbp] \\centering", "")
latex_table5 = latex_table5.replace("\\end{table}", "")

print(latex_table5)


\begin{tabular}{@{\extracolsep{5pt}}lcccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
\\[-1.8ex] & \multicolumn{2}{c}{Net Sales} & \multicolumn{2}{c}{Effective Price}  \\
\\[-1.8ex] & (1) & (2) & (3) & (4) 
 \\ & Overall & By rd & Purchase & Sales \\
\hline \\[-1.8ex]
 Treat & 0.193$^{***}$ & & -57.449$^{**}$ & 145.509$^{***}$ \\
& (0.064) & & (27.156) & (41.767) \\
 Treat - R1 & & -0.173$^{*}$ & & \\
& & (0.095) & & \\
 Treat - R2 & & 0.376$^{***}$ & & \\
& & (0.102) & & \\
 Treat - R3 & & 0.366$^{***}$ & & \\
& & (0.091) & & \\
 P-value Treat &  &  & 0.034 & 0.0 \\
 P-value Treat FWER &  &  & 0.034 & 0.0 \\
\hline \\[-1.8ex]
 Observations & 6736 & 6736 & 2014 & 1428 \\
 $R^2$ & 0.100 & 0.104 & 0.089 & 0.066 \\
 % Adjusted $R^2$ & 0.091 & 0.094 & 0.060 & 0.024 \\
 % Residual Std. Error & 1.998 & 1.994 & 639.432 & 789.099 \\
 % F Statistic & 21.288$^{***}$ & 19.708$^{***}$ & 34.249$^{***}$ & 13.002$^{***}$ \\
\hline
\hline \\[-1.8ex]
% \textit{Note:} & \multicolumn{4}{r}{$^{*}$p$<$0.1; $^{**}

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 68, but rank is 66
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 70, but rank is 68
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 62, but rank is 61
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 62, but rank is 61
  warnings.warn('covariance of constraints does not have full '


## Table 6

In [ ]:
cleanpricedata_y1y2_tab6 = cleanpricedata_y1y2.copy(deep=True)
cleanpricedata_y1y2_tab6 = cleanpricedata_y1y2_tab6[['salesPrice_trim','hi_1km_wt','hi_3km_wt','hi_5km_wt','monthnum','subloc_1km_wt_grp','subloc_3km_wt_grp','subloc_5km_wt_grp', 'in_sample','MS','lean']]
cleanpricedata_y1y2_tab6['hi'] = pd.NA
cleanpricedata_y1y2_tab6['interact'] = pd.NA
cleanpricedata_y1y2_tab6['interact_lean'] = pd.NA

In [ ]:
results = {}
for dist in ['1km_wt', '3km_wt', '5km_wt']:
    df = cleanpricedata_y1y2_tab6.copy(deep=True)
    df.dropna(subset=[f'hi_{dist}','salesPrice_trim','monthnum'], inplace=True)
    mean_price = df[(df['monthnum'] == 0) & (df[f'hi_{dist}'] == 0)]['salesPrice_trim'].mean()
    norm = 100 / mean_price

    # normalize price
    df['salesPrice_trim_norm'] = df['salesPrice_trim'] * norm

    # create hi variable
    df['hi'] = df[f'hi_{dist}']
    df['interact'] = df['monthnum'] * df['hi']

    # regression
    formula = 'salesPrice_trim_norm ~ hi + monthnum + interact'

    for ms in [1,2,3]: # 3 is pooled
        if ms == 3:
            df_filt = df[(df['in_sample'] == 1)]
        else:
            df_filt = df[(df['MS'] == ms) & (df['in_sample'] == 1)]
        model = smf.ols(formula=formula, data=df_filt).fit(cov_type='cluster', cov_kwds={'groups': df_filt[f'subloc_{dist}_grp']})
        results[(dist, ms)] = model

In [ ]:
pvals = pd.DataFrame()
# storeing pval in a df
for dv in ['hi', 'monthnum', 'interact']:
    val = {(k[0], k[1]): np.round(v.pvalues[dv],3) for k, v in results.items()}
    pvals[dv] = pd.Series(val)

# keep only columns 3km_wt and 3rd column in 1km_wt and 5km_wt
pvals = pvals.T
pvals = pvals[[('3km_wt', 1), ('3km_wt', 2), ('3km_wt', 3), ('1km_wt', 3), ('5km_wt', 3)]]

In [ ]:
def wild_bootstrap(data, model, n_bootstraps, dv, clust_var):
    """
    Wild Cluster Bootstrap-t with random signs within clusters
    """
    cluster_var = data[clust_var]
    unique_clusters = cluster_var.unique()
    boot_results = []

    for _ in range(n_bootstraps):
        boot_data = data.copy()
        # resample residuals within each cluster
        for cluster in unique_clusters:
            cluster_indices = data[cluster_var == cluster].index

            # multiply residuals by random signs, either -1 or 1, within each cluster
            signs = np.random.choice([-1, 1], size=len(cluster_indices))
            boot_data.loc[cluster_indices, dv] = model.predict(data.loc[cluster_indices]) + signs * model.resid.loc[cluster_indices]

        # Refit model on bootstrapped data
        boot_model = smf.ols(model.model.formula, data=boot_data).fit()
        boot_results.append(boot_model.params)

    return np.array(boot_results)[:,1:4]


In [ ]:
bootstrap_res = {}
bootstrap_pvals = pd.DataFrame(index=pd.MultiIndex.from_product([['1km_wt', '3km_wt', '5km_wt'], [1, 2, 3]], names=['dist', 'ms']), columns=['hi', 'monthnum', 'interact'])
n_bootstraps = 999

for dist  in ['1km_wt', '3km_wt', '5km_wt']:
    df = cleanpricedata_y1y2_tab6.copy(deep=True)
    df.dropna(subset=[f'hi_{dist}','salesPrice_trim','monthnum'], inplace=True)
    mean_price = df[(df['monthnum'] == 0) & (df[f'hi_{dist}'] == 0)]['salesPrice_trim'].mean()
    norm = 100 / mean_price

    # normalize price
    df['salesPrice_trim_norm'] = df['salesPrice_trim'] * norm
    df['salesPrice_trim_norm'] = df['salesPrice_trim_norm'].astype(float)

    # create hi variable
    df['hi'] = df[f'hi_{dist}']
    df['interact'] = df['monthnum'] * df['hi']

    # regression
    formula = 'salesPrice_trim_norm ~ hi + monthnum + interact'


    for ms in [1,2,3]: # 3 is pooled
        if ms == 3:
            df_filt = df[(df['in_sample'] == 1)]
        else:
            df_filt = df[(df['MS'] == ms) & (df['in_sample'] == 1)]
        res = wild_bootstrap(df_filt, results[(dist, ms)], n_bootstraps, 'salesPrice_trim_norm', f'subloc_{dist}_grp')
        bootstrap_res[(dist,ms)] = res

        model = results[(dist, ms)]

        for i, var in enumerate(['hi', 'monthnum', 'interact']):
            observed_coef = model.params[var]
            # calculating p-values as proportion of bootstrap coefs where abs(boot_coef) >= abs(obs_coef)
            p_value = np.round(np.mean(np.abs(bootstrap_res[(dist,ms)][:,i]) >= np.abs(observed_coef)),3)

            # store p-value in df
            bootstrap_pvals.loc[(dist,ms),var] = p_value


In [ ]:
bootstrap_pvals = bootstrap_pvals.T
bootstrap_pvals = bootstrap_pvals[[('3km_wt', 1), ('3km_wt', 2), ('3km_wt', 3), ('1km_wt', 3), ('5km_wt', 3)]]

In [ ]:
# use stargazer to create a table
result_list = [results[('3km_wt', 1)], results[('3km_wt', 2)], results[('3km_wt', 3)], results[('1km_wt', 3)], results[('5km_wt', 3)]]
stargazer = Stargazer(result_list)

# configure Stargazer object for output
stargazer.custom_columns(['Main Specification (3km)', 'Robustness (Pooled)'], [3, 2])
stargazer.rename_covariates({'hi': 'High', 'monthnum': 'Month', 'interact': 'High x Month'})
stargazer.show_degrees_of_freedom(False)
stargazer.significant_digits(3)
stargazer.covariate_order(['hi', 'monthnum', 'interact'])
# add p-values as a rows
stargazer.add_line('P-value High', pvals.loc['hi'].values.tolist())
stargazer.add_line('P-value Treat Bootstrap', bootstrap_pvals.loc['hi'].values.tolist())
stargazer.add_line('P-value Month', pvals.loc['monthnum'].values.tolist())
stargazer.add_line('P-value High Bootstrap', bootstrap_pvals.loc['monthnum'].values.tolist())
stargazer.add_line('P-value High x Month', pvals.loc['interact'].values.tolist())
stargazer.add_line('P-value Treat x High Bootstrap', bootstrap_pvals.loc['interact'].values.tolist())


latex_table6 = stargazer.render_latex()

# edit the latex table to add row for telling if Y1 Y2 or Pooled after \\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) & (7) & (8) & (9) \\
latex_table6 = latex_table6.replace("\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\",
                                "\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \n \\\ & Y1 & Y2 & Pooled & 1km & 5km \\")
latex_table6 = latex_table6.replace("Adjusted $R^2$", "% Adjusted $R^2$")
latex_table6 = latex_table6.replace("Residual Std. Error", "% Residual Std. Error")
latex_table6 = latex_table6.replace("F Statistic", "% F Statistic")
latex_table6 = latex_table6.replace("\\textit{","% \\textit{")
latex_table6 = latex_table6.replace("\\begin{table}[!htbp] \\centering", "")
latex_table6 = latex_table6.replace("\\end{table}", "")

print(latex_table6)


\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{5}{c}{% \textit{Dependent variable: salesPrice_trim_norm}} \
\cr \cline{2-6}
\\[-1.8ex] & \multicolumn{3}{c}{Main Specification (3km)} & \multicolumn{2}{c}{Robustness (Pooled)}  \\
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) 
 \\ & Y1 & Y2 & Pooled & 1km & 5km \\
\hline \\[-1.8ex]
 High & 4.410$^{**}$ & 2.855$^{}$ & 3.970$^{**}$ & 2.787$^{}$ & 3.766$^{**}$ \\
& (2.091) & (1.992) & (1.817) & (1.719) & (1.822) \\
 Month & 1.189$^{***}$ & 1.224$^{***}$ & 1.364$^{***}$ & 1.327$^{***}$ & 1.537$^{***}$ \\
& (0.363) & (0.377) & (0.350) & (0.339) & (0.291) \\
 High x Month & -0.574$^{}$ & -0.476$^{}$ & -0.573$^{}$ & -0.520$^{}$ & -0.835$^{**}$ \\
& (0.422) & (0.459) & (0.386) & (0.390) & (0.366) \\
 P-value High & 0.035 & 0.152 & 0.029 & 0.105 & 0.039 \\
 P-value Treat Bootstrap & 0.474 & 0.528 & 0.49 & 0.474 & 0.505 \\
 P-value Month & 0.001 & 0.001 & 0.0 & 0.0 & 0.0 \\
 P-value High Bootstrap & 0.484 

## Table 7

In [ ]:
# copy the raw data and create columns for treatment and interaction variable
ms1ms2_pooled_tab7 = ms1ms2_pooled.copy(deep=True)
# filter relevant columns
ms1ms2_pooled_tab7 = ms1ms2_pooled_tab7[['oafid', # id
                                         'treat12', 'treat13', 'treatMS1MS2', # treatment variables
                                         'inventory_trim', 'netrevenue_trim', 'logtotcons_trim', # outcome variables
                                         'Y1round2', 'Y1round3', 'Y2round1', 'Y2round2', 'Y2round3','hi','subloc','interviewdate']] # independent variables

ms1ms2_pooled_tab7.sort_index(inplace=True)
ms1ms2_pooled_tab7['z'] = pd.NA
ms1ms2_pooled_tab7['z_hi'] = pd.NA

### Running the first set of regressions

In [ ]:
# list of treaments
treatments = ['treat12', 'treat13', 'treatMS1MS2']

# list of dependent variables
dependent_vars = ['inventory_trim', 'netrevenue_trim', 'logtotcons_trim']

# list of changeing independent variables depending on the treatment
independent_vars = {
    'treat12': 'Y1round2 + Y1round3',
    'treat13': 'Y2round2 + Y2round3',
    'treatMS1MS2': 'Y1round2 + Y1round3 + Y2round1 + Y2round2 + Y2round3'
    }

# empty dictionary to store results
results = {}
pvals = {var: [] for var in ['z', 'hi', 'z_hi','z+z_hi']}

# Simulating the loop to replace variables and run regressions
for dv in dependent_vars:
    for treat in treatments:
        # Stata automatically omits the missing values in the regression – here we have to do it manually so we copy the data and drop variables
        df = ms1ms2_pooled_tab7.copy(deep=True)
        df = df.dropna(subset=[dv, treat, 'hi', 'subloc','interviewdate'])
        # setting treament variable
        df['z'] = df[treat] # setting z to the treatment variable

        # setting interaction variable
        df['z_hi'] = df[treat]*df['hi'] # setting z_hi to the interaction of the treatment hi saturation

        # setting the formula to run the regression
        formula = f'{dv} ~ z + hi + z_hi + interviewdate + {independent_vars[treat]}'

        # Run the regression
        model_key = f'model_{dependent_vars.index(dv)*len(treatments) + treatments.index(treat)}'
        results[model_key] = smf.ols(formula, data=df).fit(cov_type='cluster', cov_kwds={'groups': df['subloc']})
        # print(results[f'model_{i}'].summary())

        # test the hypothesis that z + z_hi = 0
        hypothesis = 'z + z_hi = 0'
        t_test = results[model_key].t_test(hypothesis)

        # store p-value round to 3 decimals
        pvals['z+z_hi'].append(np.round(t_test.pvalue,3))
        pvals['z'].append(np.round(results[model_key].pvalues['z'],3))
        pvals['hi'].append(np.round(results[model_key].pvalues['hi'],3))
        pvals['z_hi'].append(np.round(results[model_key].pvalues['z_hi'],3))

### Running boostrap regressions

In [ ]:
n_bootstraps = 999  # reported data is based on 999 iterations
bootstrap_res = {}
bootstrap_pvals = {var: [] for var in ['z', 'hi', 'z_hi']}

for dv in dependent_vars:
    for treat in treatments:
        df = ms1ms2_pooled_tab7.copy(deep=True)
        df = df.dropna(subset=[dv, treat, 'hi', 'subloc', 'interviewdate'])
        df['z'] = df[treat]
        df['z_hi'] = df[treat] * df['hi']
        df[dv] = df[dv].astype(float)

        formula = f'{dv} ~ z + hi + z_hi + interviewdate + {independent_vars[treat]}'
        model_key = f'model_{dependent_vars.index(dv)*len(treatments) + treatments.index(treat)}'
        model = results[model_key]

        # Wild bootstrap
        res = wild_bootstrap(df, model, n_bootstraps, dv,'subloc')
        bootstrap_res[model_key] = res

        for i, var in enumerate(['z', 'hi', 'z_hi']):
            observed_coef = model.params[var]
            # calculating p-values as proportion of bootstrap coefs where abs(boot_coef) >= abs(obs_coef)
            p_value = np.round(np.mean(np.abs(bootstrap_res[model_key][:,i]) >= np.abs(observed_coef)),3)
            bootstrap_pvals[var].append(p_value)

### Output to LaTeX

In [ ]:
# use stargazer to create a table
result_list = list(results.values())
stargazer = Stargazer(result_list)

# configure Stargazer object for output
stargazer.custom_columns(['Inventory', 'Net Revenues', 'Consumption'], [3, 3, 3])
stargazer.rename_covariates({'z': 'Treat', 'hi': 'High', 'z_hi': 'Treat*High'})
stargazer.show_degrees_of_freedom(False)
stargazer.significant_digits(3)
stargazer.covariate_order(['z', 'hi', 'z_hi'])
# add p-values as a rows
stargazer.add_line('P-value T + TH = 0', pvals['z+z_hi'])
stargazer.add_line('P-value Treat', pvals['z'])
stargazer.add_line('P-value Treat Bootstrap', bootstrap_pvals['z'])
stargazer.add_line('P-value High', pvals['hi'])
stargazer.add_line('P-value High Bootstrap', bootstrap_pvals['hi'])
stargazer.add_line('P-value Treat*High', pvals['z_hi'])
stargazer.add_line('P-value Treat*High Bootstrap', bootstrap_pvals['z_hi'])


latex_table7 = stargazer.render_latex()

# edit the latex table to add row for telling if Y1 Y2 or Pooled after \\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) & (7) & (8) & (9) \\
latex_table7 = latex_table7.replace("\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) & (7) & (8) & (9) \\",
                                "\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) & (7) & (8) & (9) \n \\\ & Y1 & Y2 & Pooled & Y1 & Y2 & Pooled & Y1 & Y2 & Pooled \\")
latex_table7 = latex_table7.replace("Adjusted $R^2$", "% Adjusted $R^2$")
latex_table7 = latex_table7.replace("Residual Std. Error", "% Residual Std. Error")
latex_table7 = latex_table7.replace("F Statistic", "% F Statistic")
latex_table7 = latex_table7.replace("\\textit{","% \\textit{")

print(latex_table7)

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
\\[-1.8ex] & \multicolumn{3}{c}{Inventory} & \multicolumn{3}{c}{Net Revenues} & \multicolumn{3}{c}{Consumption}  \\
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) & (7) & (8) & (9) 
 \\ & Y1 & Y2 & Pooled & Y1 & Y2 & Pooled & Y1 & Y2 & Pooled \\
\hline \\[-1.8ex]
 Treat & 0.759$^{***}$ & 0.546$^{***}$ & 0.740$^{***}$ & 1059.602$^{**}$ & 1193.768$^{*}$ & 1101.389$^{**}$ & 0.012$^{}$ & -0.051$^{}$ & -0.011$^{}$ \\
& (0.189) & (0.185) & (0.155) & (437.732) & (685.048) & (430.091) & (0.040) & (0.040) & (0.023) \\
 High & 0.124$^{}$ & -0.028$^{}$ & 0.017$^{}$ & 533.903$^{}$ & -152.603$^{}$ & 164.936$^{}$ & -0.003$^{}$ & -0.084$^{}$ & -0.047$^{}$ \\
& (0.355) & (0.219) & (0.241) & (551.179) & (558.948) & (479.685) & (0.051) & (0.053) & (0.043) \\
 Treat*High & -0.333$^{}$ & -0.065$^{}$ & -0.291$^{}$ & -1114.628$^{**}$ & -555.215$^{}$ & -816.770$^{}$ & -0.013$^{}$ & 0.174$^{***}$

## Table 8

In [ ]:
tab8_dt = ms1ms2_pooled.loc[:, ['treatMS1MS2', 'hi', 'treatMS1MS2hi', 'interviewdate', 'netrevenue_trim', 'strata_group', 'groupnum', 'subloc', 'Y1round1', 'Y1round2', 'Y1round3', 'Y2round1', 'Y2round2', 'Y2round3']].dropna()
tab8_dt['net_revenue_3'] = tab8_dt['netrevenue_trim'] * 3
model = smf.ols('net_revenue_3 ~ treatMS1MS2 + hi + treatMS1MS2hi + interviewdate + Y1round1 + Y1round2 + Y1round3 + Y2round1 + Y2round2 + Y2round3', data=tab8_dt)
results_t8 = model.fit(cov_type='cluster', cov_kwds={'groups': tab8_dt['subloc']})
model_params_t8 = results_t8.params

In [ ]:
# Annualized coefficients
treat_coef = model_params_t8['treatMS1MS2']
treat_hi_coef = (model_params_t8['treatMS1MS2'] + model_params_t8['treatMS1MS2hi'])
hi_coef = model_params_t8['hi']

# Direct beneficiary population
direct_beneficiary_pop_low = 247.0
direct_beneficiary_pop_high = 495.0

# Total direct gains
total_direct_gains_low = treat_coef * direct_beneficiary_pop_low
total_direct_gains_high = treat_hi_coef * direct_beneficiary_pop_high

# Total indirect gains (only applicable to high saturation areas)
total_indirect_gains_high = hi_coef * 3553.0

# Total gains
total_gains_low = total_direct_gains_low
total_gains_high = total_direct_gains_high + total_indirect_gains_high

# Fraction of gains direct
fraction_gains_direct_low = 1  # All gains are direct in low saturation
fraction_gains_direct_high = total_direct_gains_high / total_gains_high

# Fraction of gains indirect (only applicable to high saturation areas)
fraction_gains_indirect_high = total_indirect_gains_high / total_gains_high

table_8 = {
    "1. Direct gains/HH (Ksh)": [treat_coef, treat_hi_coef],
    "2. Indirect gains/HH (Ksh)": [0, hi_coef],
    "3. Ratio of indirect to direct gains": [0, hi_coef / treat_hi_coef],
    "4. Direct beneficiary population (HH)": [direct_beneficiary_pop_low, direct_beneficiary_pop_high],
    "5. Total local population (HH)": [3553.0, 3553.0],
    "6. Total direct gains (Ksh)": [total_direct_gains_low, total_direct_gains_high],
    "7. Total indirect gains (Ksh)": [0, total_indirect_gains_high],
    "8. Total gains (direct + indirect; Ksh)": [total_gains_low, total_gains_high],
    "9. Fraction of gains direct": [fraction_gains_direct_low, fraction_gains_direct_high],
    "10. Fraction of gains indirect": [0, fraction_gains_indirect_high],
}

# Convert the calculations to DataFrame and transpose it
table_8_df = pd.DataFrame(table_8, index=["Low Saturation", "High Saturation"]).T

# Now you can print table_8_df to see the recreated table
table_8_df

latex_table8 = table_8_df.to_latex(index=True, float_format="%.3f")
print(latex_table8)

\begin{tabular}{lrr}
\toprule
 & Low Saturation & High Saturation \\
\midrule
1. Direct gains/HH (Ksh) & 3304.166 & 853.856 \\
2. Indirect gains/HH (Ksh) & 0.000 & 494.807 \\
3. Ratio of indirect to direct gains & 0.000 & 0.579 \\
4. Direct beneficiary population (HH) & 247.000 & 495.000 \\
5. Total local population (HH) & 3553.000 & 3553.000 \\
6. Total direct gains (Ksh) & 816128.985 & 422658.745 \\
7. Total indirect gains (Ksh) & 0.000 & 1758050.851 \\
8. Total gains (direct + indirect; Ksh) & 816128.985 & 2180709.596 \\
9. Fraction of gains direct & 1.000 & 0.194 \\
10. Fraction of gains indirect & 0.000 & 0.806 \\
\bottomrule
\end{tabular}

